In [1]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import numpy as np

import sys, os; sys.path.insert(0, '../')
import BaryonForge as bfg
import pyccl as ccl

#Load cosmology object from CCL. Linear P(k) is needed since we use it for 2-halo term.
#We don't use P(k) anywhere else in this model, so it's ok to use linear P(k) throughout
cosmo = ccl.Cosmology(Omega_c = 0.26, Omega_b = 0.04, h = 0.7, sigma8 = 0.8, n_s = 0.96, matter_power_spectrum='linear')
h     = cosmo.cosmo.params.h

#Config params. Can change as you need. I store these as a dict and then unpack.
bpar_S19 = dict(theta_ej = 4, theta_co = 0.1, M_c = 1e14/h, mu_beta = 0.4,
                eta = 0.3, eta_delta = 0.3, tau = -1.5, tau_delta = 0, #Must use tau here since we go down to low mass
                A = 0.09/2, M1 = 2.5e11/h, epsilon_h = 0.015, 
                a = 0.3, n = 2, epsilon = 4, p = 0.3, q = 0.707, gamma = 2, delta = 7)

bpar_A20 = dict(alpha_g = 2, epsilon_h = 0.015, M1_0 = 2.2e11/h, 
                alpha_fsat = 1, M1_fsat = 1, delta_fsat = 1, gamma_fsat = 1, eps_fsat = 1,
                M_c = 1.2e14/h, eta = 0.6, mu = 0.31, beta = 0.6, epsilon_hydro = np.sqrt(5),
                M_inn = 3.3e13/h, M_r = 1e16, beta_r = 2, theta_inn = 0.1, theta_out = 3,
                theta_rg = 0.3, sigma_rg = 0.1, a = 0.3, n = 2, p = 0.3, q = 0.707,
                A_nt = 0.495, alpha_nt = 0.1,
                mean_molecular_weight = 0.59)

In [2]:
#Some plotting configs
plt.rc('text', usetex=True)
plt.rc('font', family='serif', size=22)
plt.rcParams["axes.linewidth"]  = 2.0
plt.rcParams["xtick.major.size"]  = 10
plt.rcParams["xtick.minor.size"]  = 5
plt.rcParams["ytick.major.size"]  = 10
plt.rcParams["ytick.minor.size"]  = 5
plt.rcParams["xtick.direction"]  = "in"
plt.rcParams["ytick.direction"]  = "in"
plt.rcParams["legend.frameon"] = 'False'
plt.rcParams['figure.figsize'] = [10, 10]

# Profile evaluations

Let's show utility of caching at the profile-level

In [3]:
a     = 1
R     = np.geomspace(1e-3, 1e1, 100) #In reality we'd eval at many more radii
M200c = np.geomspace(1e9, 1e16, 100) #This is largely representative of the masses used (in halo model calculations)

#Define the profile as usual. The cache can be done by simply
#passing a BFG profile into the CachedProfile class. This
#class simply caches results from the `real`, `fourier`, and `projected`
#functions of the class instances.
DMB        = bfg.Profiles.Schneider19.DarkMatterBaryon(**bpar_S19)
DMB_cached = bfg.utils.CachedProfile(DMB)

#We do one eval here, this will cache the result.
#This takes as long as the regular calc, which we profile below
DMB_cached.fourier(cosmo, R, M200c, a);

In [4]:
%%time

#The regular calculation is a couple of seconds (on my old laptop)
prof  = DMB.fourier(cosmo, R, M200c, a)

CPU times: user 2.05 s, sys: 141 ms, total: 2.19 s
Wall time: 1.64 s


In [5]:
%%time

#The cached version is five orders of magnitude faster, since it's just
#the read-out time.
prof  = DMB_cached.fourier(cosmo, R, M200c, a)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 196 μs


In [6]:
%%time

#But if we change the input mass limit ever so slightly, the cache key
#is different and so the calculation is redone, leading to slower runtime.
prof  = DMB_cached.fourier(cosmo, R, M200c[:98], a)

CPU times: user 2.12 s, sys: 31.2 ms, total: 2.16 s
Wall time: 1.51 s


In [7]:
#You can also confirm the cached version is identical to re-evaluating the whole thing
print(np.allclose(DMB.real(cosmo, R, M200c, a), DMB_cached.real(cosmo, R, M200c, a)))
print(np.allclose(DMB.fourier(cosmo, R, M200c, a), DMB_cached.fourier(cosmo, R, M200c, a)))

True
True


# Multicomponent/wavelength observables

The main benefit to cache-ing is when you do multi-component halomodel P(k) calculations. In this case, you are evaluating the fourier version of a profile everytime that component (e.g., Gas, Dark Matter, Stars, etc.) shows up in an auto or cross-correlation. Removing the repetition will lead to massive improvements.

The demonstration below follows largerly from Notebook 14 on HaloModel P(k) but now using caching

In [8]:
a    = 1 #Compute everything at z = 0
k    = np.geomspace(1e-3, 20, 100) #Compute across a wide range in k [1/Mpc, comoving]
rho  = ccl.rho_x(cosmo, a, 'matter', is_comoving = True)

#We will use the built-in, CCL halo model calculation tools.
HMC  = ccl.halos.halo_model.HMCalculator(mass_function = 'Tinker08', halo_bias = 'Tinker10', 
                                         mass_def = ccl.halos.massdef.MassDef200c, 
                                         log10M_min = 9, log10M_max = 16, nM = 100)


fft_precision = dict(padding_lo_fftlog = 1e-8, padding_hi_fftlog = 1e8, n_per_decade = 100)

In [9]:
par = bpar_S19

#Define the standard profiles, with various integration limits
DMO = bfg.Profiles.Schneider19.DarkMatter(**bpar_S19, r_min_int = 1e-3, r_max_int = 1e2, r_steps = 500)
GAS = bfg.Profiles.Schneider19.Gas(**bpar_S19, r_min_int = 1e-3, r_max_int = 1e2, r_steps = 500)
STR = bfg.Profiles.Schneider19.Stars(**bpar_S19, r_min_int = 1e-6, r_max_int = 5, r_steps = 500)
CLM = bfg.Profiles.Schneider19.CollisionlessMatter(**bpar_S19, max_iter = 2, reltol = 5e-2, r_steps = 500)
DMB = bfg.Profiles.Schneider19.DarkMatterBaryon(GAS, STR, CLM, DMO, bfg.Profiles.misc.Zeros(), **bpar_S19)
PRS = bfg.Profiles.Pressure(gas = GAS, darkmatterbaryon = DMB, **bpar_S19, r_min_int = 1e-4, r_max_int = 1e2, r_steps = 500)

#A function that gives us the M(r --> infty) mass of a given halo. Needed for Halomodel.
M_2_Mtot = bfg.Profiles.misc.Mdelta_to_Mtot(DMO, r_min = 1e-6, r_max = 1e2, N_int = 100)

In [10]:
#Now make a version of all profiles where we use cached versions.
DMO_cached = bfg.utils.CachedProfile(DMO)
GAS_cached = bfg.utils.CachedProfile(GAS)
STR_cached = bfg.utils.CachedProfile(STR)
CLM_cached = bfg.utils.CachedProfile(CLM)
DMB_cached = bfg.utils.CachedProfile(DMB)
PRS_cached = bfg.utils.CachedProfile(PRS)

In [11]:
#Upgrade precision of all profiles for fourier-space calculation
for p in [DMO, GAS, STR, CLM, DMB, PRS]: p.update_precision_fftlog(**fft_precision)
for p in [DMO_cached, GAS_cached, STR_cached, CLM_cached, DMB_cached, PRS_cached]: p.update_precision_fftlog(**fft_precision)

In [12]:
#We need to use an alternative HM calculator when using Schneider19 since Mtot is defined at R --> infinity.
#See Notebook 14 for more details.
HMC_flex = bfg.utils.FlexibleHMCalculator(mass_function = 'Tinker08', halo_bias = 'Tinker10', halo_m_to_mtot = M_2_Mtot,
                                          mass_def = ccl.halos.massdef.MassDef200c, 
                                          log10M_min = 9, log10M_max = 16, nM = 100)

In [13]:
%%time

#Now let's compute the auto/cross of different profiles.
#This takes 10 sec on my laptop, and most of the time is spent
#re-evaluating the profiles many times. And especially redoing the
#fourier space calculations many times.

Fiducial  = []
prof_list = [GAS, STR, CLM, DMB, PRS]
for p in range(len(prof_list)):
    for q in range(p, len(prof_list)):
        Pk = ccl.halos.pk_2pt.halomod_power_spectrum(cosmo, HMC_flex, k, a, 
                                                     prof_list[p], prof2 = prof_list[q], 
                                                     suppress_1h = lambda k : 1e-2)
        
        Fiducial.append(Pk)

/mnt/g/My Drive/OneDrive/Research/GitHub/BaryonForge/examples/../BaryonForge/Profiles/Schneider19.py:784: UserWarning: Decrease integral lower limit, r_min_int (1e-08) < minimum radius (1.0000000000000001e-11)
  warnings.warn(f"Decrease integral lower limit, r_min_int ({self.r_min_int}) < minimum radius ({np.min(r)})", UserWarning)
/mnt/g/My Drive/OneDrive/Research/GitHub/BaryonForge/examples/../BaryonForge/Profiles/Schneider19.py:786: UserWarning: Increase integral upper limit, r_max_int (100000.0) < maximum radius (2000000000.0)
  warnings.warn(f"Increase integral upper limit, r_max_int ({self.r_max_int}) < maximum radius ({np.max(r)})", UserWarning)


CPU times: user 17.5 s, sys: 93.8 ms, total: 17.6 s
Wall time: 11 s


In [14]:
%%time

#If we cache it, we get a 10x speedup. Note that we didn't pre-cache everything
#This 10x speed-up is simply from eliminating the brute-force recalculations of
#the profiles.

Cached  = []
prof_list = [GAS_cached, STR_cached, CLM_cached, DMB_cached, PRS_cached]
for p in range(len(prof_list)):
    for q in range(p, len(prof_list)):
        Pk = ccl.halos.pk_2pt.halomod_power_spectrum(cosmo, HMC_flex, k, a, 
                                                     prof_list[p], prof2 = prof_list[q], 
                                                     suppress_1h = lambda k : 1e-2)
        Cached.append(Pk)

CPU times: user 1.86 s, sys: 0 ns, total: 1.86 s
Wall time: 1.18 s


In [15]:
#The Fiducial and Cached versions are identical
np.allclose(Fiducial, Cached)

True

In [16]:
%%time

#If I run it one more time, it is dramatically faster,  since I have everything in cache now
#So now we have a 500x speedup instead. Though in practice, there shouldn't be much of a
#reason to do this kind of re-evaluation.
prof_list = [GAS_cached, STR_cached, CLM_cached, DMB_cached, PRS_cached]
for p in range(len(prof_list)):
    for q in range(p, len(prof_list)):
        Pk = ccl.halos.pk_2pt.halomod_power_spectrum(cosmo, HMC_flex, k, a, 
                                                     prof_list[p], prof2 = prof_list[q], 
                                                     suppress_1h = lambda k : 1e-2)

CPU times: user 0 ns, sys: 15.6 ms, total: 15.6 ms
Wall time: 16 ms


**NOTE:** You could also use the `TabulatedProfile` class in BaryonForge. But for halo-model calculations, the `CachedProfile` approach will be exact, whereas the Tabulated approach will incur some uncertainties (unless the the nodes of the Table are exactly/closely lined up with the masses and redshifts used for the halomodel calculation).